In [2]:
#the demodulation part of this code was submitted as an assignment for CS4020 course in 2023 in University of South Eastern Norway (USN)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, sosfilt, hilbert, spectrogram


#  band pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def bandpass_signal(signal, fs, lowcut, highcut):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(2, [low, high], btype='band', analog=False, output='sos')
    y = sosfilt(sos, signal)
    return y

#  low pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def low_signal(signal, lowcut, fs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    order = 2
    sos1 = butter(order, low, btype='low', output='sos')
    l = sosfilt(sos1, signal)
    return l

#get envelope, lowcut, fs
def process_data(data, fs, lowcut, highcut):
    
    filtered_signal = bandpass_signal(data, fs, lowcut, highcut)
    analytic_signal = hilbert(filtered_signal)
    envelope = np.abs(analytic_signal)
    processed_data = low_signal(envelope, lowcut, fs)

    return processed_data

    
# Plot the spectrogram
def plot_spectrogram(data, fs, output_path):
    f, t, Sxx = spectrogram(data, fs=fs)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Frequency [Hz]')
    plt.savefig(output_path, dpi=300)
    plt.close(fig)

def newdataset(path, date, location, first, last):
    fs = 20000
    lowcut = 2000
    highcut = 9900
    
    file_path = os.path.join(path, date)

    # Load the vibration data
    dataset = pd.read_csv(file_path, sep='\t', header=None)
    data = dataset.iloc[:, 0].values

    # get envelope, lowcut, fs
    processed_data = process_data(data, fs, lowcut, highcut)


    directory_path = r"C:\Users\zahra\nasa\filter\spectogram13"
    file_name = f"{date}_spectrogram.png"
    output_path = os.path.join(directory_path, file_name)
    

path = "C:/Users/zahra/Thesis/datasets/1st_test"
dates = os.listdir(path)
for date in dates[-465:]: #based on the previous detection step
    location = dates.index(date)
    newdataset(path, date, location, dates[0], dates[-1])
